# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [24]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [25]:
# your code here
orders = pd.read_csv("../orders.csv")
orders.head()

,Unnamed: 0,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [26]:
# your code here
agg_spend = sum(orders['amount_spent'])
agg_spend

unique_spending = orders[["CustomerID", "amount_spent"]].groupby("CustomerID").sum()[["amount_spent"]]
unique_spending.head()


,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [27]:
unique_spending["VIP"] = unique_spending["amount_spent"] > (unique_spending.quantile(q=0.95)["amount_spent"])
unique_spending.head()

,amount_spent,VIP
CustomerID,,
12346,77183.60,True
12347,4310.00,False
12348,1797.24,False
12349,1757.55,False
12350,334.40,False


In [28]:
unique_spending["Pref_C"] = (unique_spending["amount_spent"] > unique_spending["amount_spent"].quantile(q=0.75)) & (unique_spending["amount_spent"] <= (unique_spending["amount_spent"].quantile(q=0.95)))
unique_spending.head()

,amount_spent,VIP,Pref_C
CustomerID,,,
12346,77183.60,True,False
12347,4310.00,False,True
12348,1797.24,False,True
12349,1757.55,False,True
12350,334.40,False,False


In [29]:
for index,row in unique_spending.iterrows():
    if row['VIP'] == True:
        unique_spending.loc[unique_spending.index==row.index,'customer_type'] = 'VIP'
    elif row['Pref_C'] == True:
        unique_spending.loc[unique_spending.index==row.index,'customer_type'] = 'Pref_C'
    else:
        unique_spending.loc[unique_spending.index==row.index,'customer_type'] = 'STD'
        
# Dont work, lengths are not same, please help.

ValueError: Lengths must match to compare

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [45]:
merged =pd.merge(orders, unique_spending, left_on="CustomerID", right_index=True)
merged.head()

top_VIP_country = merged[merged["VIP"]==True].groupby("Country")["VIP"].count()
top_VIP_country

Country
Australia            898
Belgium               54
Channel Islands      364
Cyprus               248
Denmark               36
EIRE                7077
Finland              294
France              3290
Germany             3127
Japan                205
Netherlands         2080
Norway               420
Portugal             681
Singapore            222
Spain                511
Sweden               198
Switzerland          594
United Kingdom     84185
Name: VIP, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [43]:
# your code here
top_pref_country = merged[merged["Pref_C"]==True].groupby("Country")["Pref_C"].count()
top_pref_country

Country
Australia             130
Austria               158
Belgium              1503
Canada                135
Channel Islands       225
Cyprus                203
Denmark               181
EIRE                  161
Finland               210
France               3011
Germany              4222
Greece                 32
Iceland               182
Israel                214
Italy                 507
Japan                  67
Lebanon                45
Malta                  67
Norway                608
Poland                149
Portugal              412
Spain                1058
Sweden                 75
Switzerland           776
United Kingdom     137450
Name: Pref_C, dtype: int64

In [46]:
overall_country = top_pref_country + top_VIP_country
print(overall_country)

Country
Australia            1028.0
Austria                 NaN
Belgium              1557.0
Canada                  NaN
Channel Islands       589.0
Cyprus                451.0
Denmark               217.0
EIRE                 7238.0
Finland               504.0
France               6301.0
Germany              7349.0
Greece                  NaN
Iceland                 NaN
Israel                  NaN
Italy                   NaN
Japan                 272.0
Lebanon                 NaN
Malta                   NaN
Netherlands             NaN
Norway               1028.0
Poland                  NaN
Portugal             1093.0
Singapore               NaN
Spain                1569.0
Sweden                273.0
Switzerland          1370.0
United Kingdom     221635.0
dtype: float64
